# **Formatação e limpeza de dados - Produção Agrícola Municipal**

## **Importa bibliotecas**

In [1]:
import numpy as np
import pandas as pd

## **Importa dados como um DataFrame do Pandas**

In [2]:
raw = pd.read_csv("./data/RAW_PAM_1990-2019.zip", encoding="UTF-8", sep=";", decimal=",")
raw.head()

,Nível,Cód.,Brasil e Município,Produto das lavouras temporárias e permanentes,Ano,Variável,Valor Variável
0,BR,1,Brasil,Total,1996,Área plantada ou destinada à colheita (Hectares),46821814
1,BR,1,Brasil,Total,1996,Área plantada ou destinada à colheita - percen...,"100,00"
2,BR,1,Brasil,Total,1996,Área colhida (Hectares),45733907
3,BR,1,Brasil,Total,1996,Área colhida - percentual do total geral,"100,00"
4,BR,1,Brasil,Total,1996,Quantidade produzida (Toneladas),..


In [3]:
raw.shape

(1335360, 7)

## **1- Cria tabela auxiliar, renomeia as colunas e conserta as colunas `Variável` e `Valor Variável`**

In [8]:
columns = {
    "Cód.": "codigo_municipio",
    "Ano": "ano",
    "Área plantada ou destinada à colheita (Hectares)": "area_plantada",
    "Área colhida (Hectares)": "area_colhida",
    "Valor da produção (Mil Reais)": "valor_reais",
    "Valor da produção (Mil Cruzeiros)": "valor_cruzeiros",
    "Valor da produção (Mil Cruzeiros Reais)": "valor_cruzeiros_reais"
}

df = pd.DataFrame(columns=list(columns.keys()))

In [5]:
mun_ids = raw["Cód."].unique()
mun_ids_total = len(mun_ids)
counter = 0
print("Total de municípios >>", len(mun_ids))
for municipio in mun_ids:
    counter += 1
    print(f"\r{counter}/{mun_ids_total}", end="")
    for ano in raw["Ano"].unique():
        temp = raw.loc[(raw["Cód."] == municipio) & (raw["Ano"] == ano)]
        temp_row = temp.iloc[0]
        dic = {
            "Cód.": municipio,
            "Ano": ano,
            "Área plantada ou destinada à colheita (Hectares)": "",
            "Área colhida (Hectares)": "",
            "Valor da produção (Mil Reais)": "",
            "Valor da produção (Mil Cruzeiros)": "",
            "Valor da produção (Mil Cruzeiros Reais)": ""
        }
        for _, row in temp.iterrows():
            dic[str(row["Variável"])] = row["Valor Variável"]
        df = df.append(dic, ignore_index=True)
df = df.rename(columns)
df = df.sort_values(by=["ano", "codigo_municipio"]).reset_index()
df["area_plantada"] = df["area_plantada"].replace(["...", "-"], np.NaN)
df["area_colhida"] = df["area_colhida"].replace(["...", "-"], np.NaN)
df["valor_reais"] = df["valor_reais"].replace(["...", "-"], np.NaN)
df["valor_cruzeiros"] = df["valor_cruzeiros"].replace(["...", "-"], np.NaN)
df["valor_cruzeiros_reais"] = df["valor_cruzeiros_reais"].replace(["...", "-"], np.NaN)
df["razao_plantio_colheita"] = df["area_colhida"]/df["area_plantada"]

Total de municípios >> 5564
5564/5564

KeyError: 'ano'

In [25]:
df

,codigo_municipio,ano,area_plantada,area_colhida,valor_reais,valor_cruzeiros,valor_cruzeiros_reais
0,1,1990,53152446,50514696,,1722709443,
1,1,1991,51753078,50643873,,8740093676,
2,1,1992,52273579,50714599,,103777476104,
3,1,1993,49226752,46629302,,,2111804269
4,1,1994,52815030,51354033,27500582,,
...,...,...,...,...,...,...,...
166915,5300108,2015,161011,160622,646383,,
166916,5300108,2016,166489,166419,764001,,
166917,5300108,2017,162894,162894,822413,,
166918,5300108,2018,165073,164900,848687,,


In [27]:
df = df[list(columns.values())].rename(columns=columns)
df = df.sort_values(by=["codigo_municipio", "ano"]).reset_index(drop=True)
df["area_plantada"] = df["area_plantada"].replace(["...", "-"], np.NaN)
df["area_colhida"] = df["area_colhida"].replace(["...", "-"], np.NaN)
df["valor_reais"] = df["valor_reais"].replace(["...", "-"], np.NaN)
df["valor_cruzeiros"] = df["valor_cruzeiros"].replace(["...", "-"], np.NaN)
df["valor_cruzeiros_reais"] = df["valor_cruzeiros_reais"].replace(["...", "-"], np.NaN)
df["razao_plantio_colheita"] = df["area_colhida"].astype(float)/df["area_plantada"].astype(float)

In [28]:
df.head()

,codigo_municipio,ano,area_plantada,area_colhida,valor_reais,valor_cruzeiros,valor_cruzeiros_reais,razao_plantio_colheita
0,1,1990,53152446,50514696,,1722709443,,0.950374
1,1,1991,51753078,50643873,,8740093676,,0.978567
2,1,1992,52273579,50714599,,103777476104,,0.970177
3,1,1993,49226752,46629302,,,2111804269,0.947235
4,1,1994,52815030,51354033,27500582,,,0.972337


In [29]:
df.to_csv("./data/PAM.zip", sep=";", index=False, compression=dict(method="zip", archive_name="PAM.csv"))